In [4]:
import pandas as pd
import torch
import numpy as np
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer

In [5]:
# --- 1. CARREGAR O MODELO E OS DADOS NÃO ROTULADOS ---

# Carregar o modelo_v2 que acabamos de treinar
model_path = "/Users/giossaurus/Developer/leia_tcc/models/leia_classifier_v2_final"
model = AutoModelForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Carregar a amostra de trabalho completa e os dados já anotados
df_working_set = pd.read_csv('/Users/giossaurus/Developer/leia_tcc/data/processed/amostra_trabalho_1000.csv')
df_seed_set = pd.read_csv('/Users/giossaurus/Developer/leia_tcc/data/processed/seed_set_anotado.csv')
df_cycle1 = pd.read_csv('/Users/giossaurus/Developer/leia_tcc/data/processed/active_learning_cycle1.csv')

# Combinar os IDs de tudo que já foi anotado
annotated_ids = pd.concat([
    df_seed_set['question_id'],
    df_cycle1['question_id']
]).unique()

# Filtrar para obter apenas os dados que AINDA NÃO foram anotados
unlabeled_df = df_working_set[~df_working_set['question_id'].isin(annotated_ids)].copy()
unlabeled_df = unlabeled_df.rename(columns={'question': 'text'})

print(f"Encontrados {len(unlabeled_df)} exemplos não rotulados para análise no Ciclo 3.")

# Converter para o formato do Hugging Face
unlabeled_dataset = Dataset.from_pandas(unlabeled_df)

Encontrados 751 exemplos não rotulados para análise no Ciclo 3.


In [6]:
# --- 2. FAZER PREDIÇÕES E CALCULAR A INCERTEZA ---

# Tokenizar os dados não rotulados
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

tokenized_unlabeled_dataset = unlabeled_dataset.map(tokenize_function, batched=True)

# Criar um 'Trainer' apenas para fazer predições
trainer = Trainer(model=model)

# Obter as predições (logits) do modelo
predictions = trainer.predict(tokenized_unlabeled_dataset)
logits = predictions.predictions

# Converter logits para probabilidades
probabilities = torch.nn.functional.softmax(torch.from_numpy(logits), dim=-1).numpy()

# Calcular a incerteza (menor margem entre as duas classes mais prováveis)
sorted_probabilities = np.sort(probabilities, axis=1)
margin = sorted_probabilities[:, -1] - sorted_probabilities[:, -2]
unlabeled_df['uncertainty'] = 1 - margin

Map:   0%|          | 0/751 [00:00<?, ? examples/s]

/opt/homebrew/Caskroom/miniforge/base/envs/torch-mps/lib/python3.10/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


In [7]:
# --- 3. EXIBIR AS PERGUNTAS PARA A PRÓXIMA ANOTAÇÃO ---

# Ordenar o dataframe pela incerteza e pegar as 50 mais incertas
N_SAMPLES_TO_ANNOTATE = 50
df_to_annotate_c3 = unlabeled_df.sort_values(by='uncertainty', ascending=False).head(N_SAMPLES_TO_ANNOTATE)

print(f"\n--- AS {N_SAMPLES_TO_ANNOTATE} PERGUNTAS MAIS INCERTAS (CICLO 3) PARA VOCÊ ANOTAR: ---")

for index, row in df_to_annotate_c3.iterrows():
    print(f"--- Pergunta ID: {row['question_id']} (Incerteza: {row['uncertainty']:.2f}) ---")
    print(row['text'])
    print("-" * 50)

# Preparar dados para Label Studio com o campo 'question' correto
df_for_label_studio_c3 = df_to_annotate_c3.copy()
df_for_label_studio_c3['question'] = df_for_label_studio_c3['text']

# Salvar este lote para anotação no Label Studio
df_for_label_studio_c3.to_csv('/Users/giossaurus/Developer/leia_tcc/data/processed/active_learning_cycle3_to_annotate.csv', index=False)
print(f"\nArquivo com as {N_SAMPLES_TO_ANNOTATE} perguntas salvo em 'data/processed/active_learning_cycle3_to_annotate.csv'")


--- AS 50 PERGUNTAS MAIS INCERTAS (CICLO 3) PARA VOCÊ ANOTAR: ---
--- Pergunta ID: 93f94433a5643adbe392b3a682ddf9b4746b998837f1aab2a13f026751ff92400e0bc058a9d1d4578173ff3607add121954c76678ae891a1b1050e9cc4b79dd2 (Incerteza: 1.00) ---
TEXTO DE REFERÊNCIA: TEXTO I

Heitor dos Prazeres começou a se dedicar à pintura no final dos anos 1930, quando já havia se estabelecido como compositor de samba no Rio de Janeiro. Ele representou muitos dos lugares que frequentava, entre eles gafieiras, bares boêmios, blocos de Carnaval e terreiros. Sua obra é atravessada por um ritmo conferido tanto pelo movimento sugerido dos corpos, que em geral aparecem dançando ou em transe, como pelo contraste das cores. Por ser autodidata e pelo repertório visual que consolidou em sua carreira, Prazeres foi muitas vezes categorizado como um artista naïf. Essa denominação, no entanto, parece ofuscar a complexidade de seu trabalho.

TEXTO II

Heitor dos Prazeres, Carnaval, s.d., óleo sobre madeira, 37 cm × 84,50 cm.